### monthly revenue mart

In [0]:
%sql
-- To track how much revenue is generated each month
create or replace table nyc_taxi.gold.monthly_revenue_mart as
select
  vendor_name,
  pickup_year,
  pickup_month,
  count(*) as total_trips,
  round(sum(fare_amount),2) AS total_fare,
  round(sum(tip_amount),2) AS total_tips,
  round(sum(total_amount),2) AS total_revenue
FROM nyc_taxi.gold.fact_trips
join nyc_taxi.gold.dim_vendor on fact_trips.vendor_id = dim_vendor.vendor_id
where pickup_year between 2021 and 2025
GROUP BY vendor_name,pickup_year, pickup_month
order by vendor_name,pickup_year, pickup_month;

### average speed by hour mart

In [0]:
%sql
--- To analyze trip speed trends by hour to identify traffic patterns
CREATE OR REPLACE TABLE nyc_taxi.gold.avg_speed_by_hour_mart AS
SELECT
  pickup_hour,
  ROUND(AVG(trip_speed_mph), 2) AS avg_speed_mph,
  COUNT(*) AS num_trips
FROM nyc_taxi.gold.fact_trips where trip_speed_mph BETWEEN 0 AND 150
GROUP BY pickup_hour
ORDER BY pickup_hour;

### Zone to Zone Trip Volume Mart

In [0]:
%sql
CREATE OR REPLACE TABLE nyc_taxi.gold.zone_to_zone_trip_volume_mart AS
SELECT
    f.pickup_year,
    f.pickup_month,
    pu.borough AS pickup_borough,
    do.borough AS dropoff_borough,
    t.trip_type_description,
    COUNT(*) AS total_trips,
    ROUND(AVG(f.passenger_count), 2) AS avg_passenger_count
FROM nyc_taxi.gold.fact_trips f
JOIN nyc_taxi.gold.dim_location pu ON f.pickup_location_id = pu.location_id
JOIN nyc_taxi.gold.dim_location do ON f.dropoff_location_id = do.location_id
JOIN nyc_taxi.gold.dim_trip_type t ON f.trip_type = t.trip_type
where pickup_year between 2021 and 2025 and pu.borough not in ('Unknown','N/A') and do.borough not in ('Unknown','N/A')
GROUP BY
    f.pickup_year,
    f.pickup_month,
    t.trip_type_description,
    pu.borough,
    do.borough
ORDER BY total_trips desc;

### Weekday vs Weekend Trip Patterns

In [0]:
%sql
--- To compare trip behavior across weekdays and weekends
CREATE OR REPLACE TABLE nyc_taxi.gold.weekday_vs_weekend_mart AS
SELECT
  CASE 
    WHEN pickup_dayofweek IN (1, 7) THEN 'Weekend'
    ELSE 'Weekday'
  END AS day_type,
  COUNT(*) AS trip_count,
  ROUND(AVG(trip_distance), 2) AS avg_distance,
  ROUND(AVG(total_amount), 2) AS avg_revenue
FROM nyc_taxi.gold.fact_trips
GROUP BY day_type;